In [1]:
pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requir

In [2]:
pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 80.6 MB/s eta 0:00:00


In [3]:
!pip install optimum onnxruntime onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 77.4 MB/s eta 0:00:00


In [4]:
!pip install cupy-cuda11x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 MB 8.5 MB/s eta 0:00:00


In [7]:
import pandas as pd
from google.colab import files

In [8]:
uploaded = files.upload()
df = pd.read_csv('Silver/data_pretraité.csv')

Saving data_pretraité.csv to data_pretraité.csv


In [12]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import CamembertTokenizer, pipeline


/usr/local/lib/python3.10/dist-packages/cupy/_environment.py:540: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [10]:
HUB_MODEL = "cmarkea/distilcamembert-base-sentiment"

In [13]:
tokenizer = CamembertTokenizer.from_pretrained(HUB_MODEL)
quantized_model = ORTModelForSequenceClassification.from_pretrained(HUB_MODEL, file_name="model.onnx")
onnx_qa_quantized = pipeline("text-classification", model=quantized_model, tokenizer=tokenizer)

model.onnx:   0%|          | 0.00/272M [00:00<?, ?B/s]

In [14]:
df['prediction'] = df['Commentaire'].apply(lambda x: onnx_qa_quantized(x)[0]['label'])
df['score'] = df['Commentaire'].apply(lambda x: onnx_qa_quantized(x)[0]['score'])

In [15]:
df.head()

,Auteur,Date,Commentaire,Note,prediction,score
0,JULIE,2024-12-11,agreables a porter et bon amorti,5,4 stars,0.598047
1,PIERRETTE,2024-12-11,bon produit,5,4 stars,0.577293
2,VANESSA,2024-12-10,bon rapport qualite prix,5,4 stars,0.599083
3,JEAN CHARLES,2024-12-10,tiens bien au pied,5,4 stars,0.470387
4,BARBARA,2024-12-10,utilisee pour la salle de sport et sur tapis d...,5,4 stars,0.492338


In [16]:
df.to_csv('Silver/prediction.csv', index=False, encoding='utf-8')